## Test accuracy on 20 CIFAR-100 classes using DenseNet121

### Import all the necessary libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras import layers, models
import numpy as np
from tensorflow.keras.datasets import cifar100

2025-05-24 20:19:59.003934: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-24 20:19:59.369855: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-24 20:19:59.371936: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-24 20:20:00.793509: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


###  Load CIFAR-100 fine labels (100-class)

In [2]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')

### Choose 20 classes

In [3]:
selected_classes = list(range(20))
train_mask = np.isin(y_train, selected_classes).flatten()
test_mask = np.isin(y_test, selected_classes).flatten()

x_train_20 = x_train[train_mask]
y_train_20 = y_train[train_mask]
x_test_20 = x_test[test_mask]
y_test_20 = y_test[test_mask]

### Map label to 0 - 19

In [4]:
label_map = {cls: i for i, cls in enumerate(selected_classes)}
y_train_20 = np.array([label_map[y[0]] for y in y_train_20])
y_test_20 = np.array([label_map[y[0]] for y in y_test_20])

### Resize to 128x128

In [5]:
x_train_20 = tf.image.resize(x_train_20, (128, 128)).numpy()
x_test_20 = tf.image.resize(x_test_20, (128, 128)).numpy()

2025-05-24 20:20:03.190435: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-05-24 20:20:03.191020: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2025-05-24 20:20:03.248359: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1966080000 exceeds 10% of free system memory.


### Preprocess

In [6]:
x_train_20 = preprocess_input(x_train_20)
x_test_20 = preprocess_input(x_test_20)

### Converts Label

In [7]:
y_train_20_cat = tf.keras.utils.to_categorical(y_train_20, 20)
y_test_20_cat = tf.keras.utils.to_categorical(y_test_20, 20)

### Load DenseNet121 base model

In [8]:
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
base_model.trainable = False  # freeze base

# 8. Build model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(20, activation='softmax')
])

# 9. Compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 10. Train
model.fit(x_train_20, y_train_20_cat, epochs=5, batch_size=64, validation_split=0.1)

# 11. Evaluate
loss, acc = model.evaluate(x_test_20, y_test_20_cat)
print("Test accuracy on 20 CIFAR-100 classes using DenseNet121:", acc)

Epoch 1/5


2025-05-24 20:20:10.847092: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1769472000 exceeds 10% of free system memory.


141/141 [==============================] - 126s 862ms/step - loss: 1.1714 - accuracy: 0.6499 - val_loss: 0.5946 - val_accuracy: 0.7990
Epoch 2/5
141/141 [==============================] - 120s 854ms/step - loss: 0.5750 - accuracy: 0.8112 - val_loss: 0.5571 - val_accuracy: 0.8160
Epoch 3/5
141/141 [==============================] - 120s 849ms/step - loss: 0.4596 - accuracy: 0.8451 - val_loss: 0.5143 - val_accuracy: 0.8270
Epoch 4/5
141/141 [==============================] - 120s 851ms/step - loss: 0.3771 - accuracy: 0.8718 - val_loss: 0.5054 - val_accuracy: 0.8330
Epoch 5/5
63/63 [==============================] - 22s 357ms/step - loss: 0.4266 - accuracy: 0.8670
Test accuracy on 20 CIFAR-100 classes using DenseNet121: 0.8669999837875366
